In [48]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import date
from pandas.tseries.offsets import BDay
import requests
import lxml.html as lh
import pandas as pd
import os
import datetime as dt

def b3sa_Options(token):
    r = requests.get(token)
    vJson = json.loads(r.content)
    vJson['token']
    optionList = 'https://arquivos.b3.com.br/api/download/?token={}'.format(vJson['token'])
    r1 = requests.get(optionList)
    data = r1.content.decode("ISO-8859-1").splitlines()
    df = pd.DataFrame(data) 
    df.dropna(inplace = True) 
    df = pd.DataFrame(df.applymap(str))
    df = df[0].str.split(";", n = 100, expand = True)
    df.columns = df.iloc[0]
    df = df[1:]
    df = df.reset_index()
    df.drop("index", axis = 1, inplace=True)
    return df

if dt.datetime.fromtimestamp(os.path.getctime("b3.csv")).strftime("%Y-%m-%d") != date.today().strftime("%Y-%m-%d"):
    token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=InstrumentsConsolidated&date={}&recaptchaToken='.format(date.today().strftime("%Y-%m-%d"))
    df=b3sa_Options(token)
    df2 = df[["TckrSymb", "Asst", "SgmtNm", "SctyCtgyNm", "XprtnDt", "TradgEndDt", "OptnTp", "ExrcPric", "OptnStyle"]]
    df2 = df2[df2['SctyCtgyNm']=='OPTION ON EQUITIES']
    df2['ExrcPric'] = [x.replace(',', '.') for x in df2['ExrcPric']]
    df2['ExrcPric'] = df2['ExrcPric'].astype(float)

    lastBD = pd.datetime.today() - BDay(1)
    lastBD = lastBD.strftime("%Y-%m-%d")
    token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=TradeInformationConsolidated&date={}'.format(lastBD)
    df_trade=b3sa_Options(token)

    df_trade = df_trade[(df_trade['SgmtNm']=='EQUITY PUT') | (df_trade['SgmtNm']=='EQUITY CALL')]
    df_trade=df_trade[(df_trade['TradQty']!='')]
    df_trade["Strike"]=0.0
    df_trade["Ticker"]=""
    df_trade["Validade"]=""
    df_trade["Tipo"]=""
    df_trade["Price"] = 0.0

    for i in df_trade.TckrSymb:
        try:
            df_trade.loc[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Strike'] = df2.loc[df2["TckrSymb"]==i].ExrcPric.values[0]
            df_trade.loc[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Ticker'] = df2.loc[df2["TckrSymb"]==i].Asst.values[0]
            df_trade.loc[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Validade'] = df2.loc[df2["TckrSymb"]==i].XprtnDt.values[0]
            df_trade.loc[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Tipo'] = df2.loc[df2["TckrSymb"]==i].OptnStyle.values[0]
        except:
            pass
    if df_trade.shape[0] > 100:
        df_trade.to_csv("b3.csv",index=False)
        print(1)
elif dt.datetime.fromtimestamp(os.path.getctime("b3.csv")).strftime("%Y-%m-%d") == date.today().strftime("%Y-%m-%d"):
    df_trade=pd.read_csv("b3.csv")
    print(2)
else:
    print("Error, dataset not loaded as expected")


2


In [50]:
url="https://www.iqb3.com.br/iq/Todas"
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
tr_elements=tr_elements[1:]
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
    
    
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=13:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Dict={title:column for (title,column) in col}
df_iq=pd.DataFrame(Dict)

In [56]:
vTopacoes

,0,Atual
0,PETR4,22.89
1,VALE3,58.59


In [57]:
vTopacoes = pd.read_excel(r"C:\Users\kiyo_\Desktop\projects\options\Cotacao.xlsm")
for i in df_trade.Ticker:
    try:
        df_trade.loc[df_trade.loc[df_trade['Ticker']==i].index.values,'Price'] = vTopacoes.loc[vTopacoes[0]==i].Atual.values[0]
    except:
        pass

df_trade = df_trade[df_trade["Price"]!=0.0]
df_Call = df_trade[df_trade["SgmtNm"]=="EQUITY CALL"]
df_Put = df_trade[df_trade["SgmtNm"]=="EQUITY PUT"]

#df_Call=df_Call[(df_Call["Price"]/df_Call["Strike"]>1.08) & (df_Call["Price"]/df_Call["Strike"]<1.15)]
#df_Put=df_Put[(df_Put["Price"]/df_Put["Strike"]>0.85) & (df_Put["Price"]/df_Put["Strike"]<0.92)]

df_Put=df_Put[(df_Put["Price"]/df_Put["Strike"]>1.08) & (df_Put["Price"]/df_Put["Strike"]<1.15)]
df_Call=df_Call[(df_Call["Price"]/df_Call["Strike"]>0.85) & (df_Call["Price"]/df_Call["Strike"]<0.92)]

df_2p2c=df_Call.append(df_Put)
df_2p2c["MinPric"]=""
df_2p2c.rename(columns={"MinPric": "IQ"}, inplace=True)

C:\Users\kiyo_\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [69]:
try:
    for j in df_2p2c.TckrSymb:
        df_2p2c.loc[df_2p2c.loc[df_2p2c['TckrSymb']==j].index.values,'IQ'] = df_iq.loc[df_iq["Papel"]==j].IQ.values[0]
except:
    pass
    
new_df1=df_2p2c[["Ticker","TckrSymb","SgmtNm","Tipo","Validade","TradQty","FinInstrmQty","Strike","Price","IQ"]]
new_df1=new_df1.reset_index()
new_df1=new_df1.drop(['index'], axis=1)
new_df1["Validade"] = pd.to_datetime(new_df1["Validade"])
new_df1["Days"] = new_df1["Validade"].apply(lambda x: x - pd.datetime.today())
new_df1["Days"] = new_df1["Days"].apply(lambda x: x.days)
new_df1["TradQty"]=new_df1["TradQty"].astype(int)
new_df1 = new_df1[new_df1["TradQty"]>3]
new_df1 = new_df1[["Ticker","TckrSymb","SgmtNm","Validade","Tipo","Days","Strike","IQ"]]
new_df1["Ticker1"] = new_df1["Ticker"].apply(lambda x: "=@BULLDDE|MOFV!"+x)
new_df1["TckrSymb1"] = new_df1["TckrSymb"].apply(lambda x: "=@BULLDDE|MOFC!"+x)
new_df1['IQ'] = [x.replace(',', '.') for x in new_df1['IQ']]
new_df1.to_csv("Options_V3.csv")

(58, 10)